<a href="https://colab.research.google.com/github/ngamzedeniz/DS-Miuul/blob/main/rfm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import datetime as dt
import pandas as pd

import matplotlib.pyplot as plt



# pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.float_format', lambda x: '%.4f' % x)

# Adım 1
df_ = pd.read_csv("/content/flo_data_20k.csv")
df = df_.copy()

#Adım 2
df.head(10)
df.columns
df.describe().T
df.isnull().sum()
df.sum()

# Adım 3

df["order_num_total"] = df["order_num_total_ever_online"] + df["order_num_total_ever_offline"]
df["customer_value_total"] = df["customer_value_total_ever_online"] + df["customer_value_total_ever_offline"]

# Adım 4
df["last_order_date"].max()
today_date = dt.datetime(2021, 6, 3)
from datetime import datetime

date_columns = df.columns[df.columns.str.contains("date")]
df[date_columns] = df[date_columns].apply(pd.to_datetime, format='%Y-%m-%d')



rfm = df.groupby("master_id").agg({"last_order_date": lambda date: (today_date - date.iloc[0]).days,
                                   "order_num_total": lambda num: num,
                                   "customer_value_total": lambda value: value,
                                   })


In [4]:
df["last_order_date"].max()

Timestamp('2021-05-30 00:00:00')

In [6]:
rfm.head()

,last_order_date,order_num_total,customer_value_total
master_id,,,
00016786-2f5a-11ea-bb80-000d3a38a36f,12,5.0000,776.0700
00034aaa-a838-11e9-a2fc-000d3a38a36f,300,3.0000,269.4700
000be838-85df-11ea-a90b-000d3a38a36f,215,4.0000,722.6900
000c1fe2-a8b7-11ea-8479-000d3a38a36f,29,7.0000,874.1600
000f5e3e-9dde-11ea-80cd-000d3a38a36f,22,7.0000,1620.3300


In [7]:
rfm.columns = ["recency","frequency","monetary"]

In [11]:
# Adım6:  Enfazlakazancıgetirenilk 10 müşteriyisıralayınız.
rfm["monetary"].sort_values(ascending=False).head(10)

master_id
5d1c466a-9cfd-11e9-9897-000d3a38a36f   45905.1000
d5ef8058-a5c6-11e9-a2fc-000d3a38a36f   36818.2900
73fd19aa-9e37-11e9-9897-000d3a38a36f   33918.1000
7137a5c0-7aad-11ea-8f20-000d3a38a36f   31227.4100
47a642fe-975b-11eb-8c2a-000d3a38a36f   20706.3400
a4d534a2-5b1b-11eb-8dbd-000d3a38a36f   18443.5700
d696c654-2633-11ea-8e1c-000d3a38a36f   16918.5700
fef57ffa-aae6-11e9-a2fc-000d3a38a36f   12726.1000
cba59206-9dd1-11e9-9897-000d3a38a36f   12282.2400
fc0ce7a4-9d87-11e9-9897-000d3a38a36f   12103.1500
Name: monetary, dtype: float64

In [12]:
#Adım7:  Enfazlasiparişiverenilk 10 müşteriyisıralayınız.
rfm["frequency"].sort_values(ascending=False).head(10)

master_id
5d1c466a-9cfd-11e9-9897-000d3a38a36f   202.0000
cba59206-9dd1-11e9-9897-000d3a38a36f   131.0000
a57f4302-b1a8-11e9-89fa-000d3a38a36f   111.0000
fdbe8304-a7ab-11e9-a2fc-000d3a38a36f    88.0000
329968c6-a0e2-11e9-a2fc-000d3a38a36f    83.0000
73fd19aa-9e37-11e9-9897-000d3a38a36f    82.0000
44d032ee-a0d4-11e9-a2fc-000d3a38a36f    77.0000
b27e241a-a901-11e9-a2fc-000d3a38a36f    75.0000
d696c654-2633-11ea-8e1c-000d3a38a36f    70.0000
a4d534a2-5b1b-11eb-8dbd-000d3a38a36f    70.0000
Name: frequency, dtype: float64

In [16]:
rfm["recency_score"] = pd.qcut(rfm["recency"], 5, labels = [5, 4, 3, 2, 1])
rfm["monetary_score"] = pd.qcut(rfm["monetary"], 5, labels = [1, 2, 3, 4, 5])
rfm["frequency_score"] = pd.qcut(rfm["frequency"].rank(method = "first"), 5, labels = [1, 2, 3, 4, 5])

In [18]:
rfm["RF_SCORE"] = rfm["recency_score"].astype(str) + rfm["frequency_score"].astype(str)

In [19]:
seg_map = {
    r'[1-2][1-2]': 'Hibernating',
    r'[1-2][3-4]': 'At Risk',
    r'[1-2]5': 'Can\'t Lose',
    r'3[1-2]': 'About to Sleep',
    r'33': 'Need Attention',
    r'[3-4][4-5]': 'Loyal Customers',
    r'41': 'Promising',
    r'51': 'New Customers',
    r'[4-5][2-3]': 'Potential Loyalists',
    r'5[4-5]': 'Champions'
}

In [20]:
rfm["segment"] = rfm["RF_SCORE"].replace(seg_map, regex = True)

In [21]:
rfm[["segment","recency","frequency","monetary"]].groupby("segment").agg(["mean","count"])

recency       frequency        monetary      
                        mean count      mean count      mean count
segment                                                           
About to Sleep      116.0316  1643    2.4066  1643  361.6494  1643
At Risk             244.3290  3152    4.4702  3152  648.3250  3152
Can't Lose          237.1591  1194   10.7169  1194 1481.6524  1194
Champions            19.1422  1920    8.9651  1920 1410.7089  1920
Hibernating         249.4263  3589    2.3915  3589  362.5833  3589
Loyal Customers      84.5579  3375    8.3564  3375 1216.2572  3375
Need Attention      115.0372   806    3.7395   806  553.4366   806
New Customers        19.9762   673    2.0000   673  344.0495   673
Potential Loyalists  38.8697  2925    3.3108  2925  533.7413  2925
Promising            60.6946   668    2.0000   668  334.1533   668

In [26]:
df["interested_in_categories_12"].str.contains("KADIN")

0         True
1         True
2         True
3        False
4        False
         ...  
19940    False
19941    False
19942    False
19943    False
19944     True
Name: interested_in_categories_12, Length: 19945, dtype: bool

In [28]:
rfm[(rfm["segment"]=="Champions") & (rfm["segment"]=="Loyal Customers")].head()


,recency,frequency,monetary,recency_score,monetary_score,frequency_score,RFM_SCORE,RF_SCORE,segment
master_id,,,,,,,,,
